In [1]:
import syft as sy

In [2]:
from syft.core.store.storeable_object import StorableObject

In [3]:
from syft.core.common import UID

In [4]:
import torch

# test functions

In [5]:
from syft.core.common.serde.serialize import _serialize
from syft.core.common.serde.deserialize import _deserialize
def serde(data, via_proto=True):
    to_proto = True if via_proto else False
    to_bytes = not to_proto
    from_proto = to_proto
    from_bytes = to_bytes
    
    data_serde = _deserialize(
        blob=_serialize(
            obj=data,
            to_proto=to_proto,
            to_bytes=to_bytes
        ),
        from_proto=from_proto,
        from_bytes=from_bytes
    )
    
    return data_serde

In [6]:
import math
def assert_eq(obj_1, obj_2):
    if isinstance(obj_1, torch.Tensor):
        assert (obj_1==obj_2).any()
    elif isinstance(obj_1, sy.lib.python.Float):
        assert math.isclose(obj_1, obj_2, rel_tol=1e-6)
    else:
        assert obj_1==obj_2        

In [7]:
def test_serde(data, via_proto=True):
    data_serde = serde(data, via_proto=via_proto)
    assert_eq(data_serde, data)

In [8]:
def test_StorableObject_data(data, via_proto=True):
    storable = StorableObject(id=UID(), data=data)
    storable_serde = serde(storable, via_proto=via_proto)
    assert_eq(storable_serde.data, data)

In [9]:
remote = sy.VirtualMachine().get_root_client()
def test_send_get(data, remote=remote):
    tags = ["tag#"]
    description = "test description"
    ptr = data.send(remote, tags=tags, description=description, searchable=True)
    assert remote.store[-1].tags==ptr.tags==tags
    assert remote.store[-1].description==ptr.description==description
    assert remote.store[-1].id_at_location==ptr.id_at_location
    assert_eq(ptr.get(), data)

In [10]:
def test(data):
    test_serde(data, via_proto=True)
    test_serde(data, via_proto=False)
    
    test_StorableObject_data(data, via_proto=True)
    test_StorableObject_data(data, via_proto=False)
    
    test_send_get(data)

# Int

In [11]:
data = sy.lib.python.Int(77)
test(data)

# torch.device

In [12]:
data = torch.device('cpu')
test(data)

# String

In [13]:
data = sy.lib.python.String("Hello world")
test(data)

# Dict

In [14]:
data = sy.lib.python.Dict({
    "a": 1,
    "b": 2
})
test(data)

# Tensor

In [15]:
data = torch.rand((2,3))
test(data)

# Bool

In [16]:
data = sy.lib.python.Bool(True)
test(data)

# Float

In [17]:
data = sy.lib.python.Float(1.1)
test(data)

# PSI

In [45]:
!pytest ../../../../tests/syft/lib/psi/psi_test.py

============================= test session starts ==============================
platform linux -- Python 3.8.5, pytest-6.1.2, py-1.9.0, pluggy-0.13.1 -- /mnt/d/ubuntu/anaconda3/envs/pysyft_dev/bin/python
cachedir: .pytest_cache
benchmark: 3.2.3 (defaults: timer=time.perf_counter disable_gc=False min_rounds=5 min_time=0.000005 max_time=1.0 calibration_precision=10 warmup=False warmup_iterations=100000)
rootdir: /mnt/d/github/PySyft, configfile: setup.cfg
plugins: asyncio-0.14.0, benchmark-3.2.3, cov-2.10.1, forked-1.3.0, xdist-2.1.0, typeguard-2.10.0
collected 4 items                                                              

../../../../tests/syft/lib/psi/psi_test.py::test_psi[True-True] PASSED   [ 25%]
../../../../tests/syft/lib/psi/psi_test.py::test_psi[True-False] PASSED  [ 50%]
../../../../tests/syft/lib/psi/psi_test.py::test_psi[False-True] PASSED  [ 75%]
../../../../tests/syft/lib/psi/psi_test.py::test_psi[False-False] PASSED [100%]

----------- coverage: platform linux, pyt

/mnt/d/github/PySyft/src/syft/core/store/store_interface.py                                                      77     17    78%   30, 40, 50, 60, 70, 83, 99, 114, 118, 137, 144, 148, 152, 156, 160, 164, 179
/mnt/d/github/PySyft/src/syft/core/store/store_memory.py                                                         70     14    80%   46, 50, 54, 66, 72-74, 87-89, 93, 96, 100, 103
/mnt/d/github/PySyft/src/syft/core/store/storeable_object.py                                                    150     30    80%   87, 155-158, 213, 225-228, 234, 238, 269, 277, 281-305, 309
/mnt/d/github/PySyft/src/syft/decorators/__init__.py                                                              2      0   100%
/mnt/d/github/PySyft/src/syft/decorators/syft_decorator_impl.py                                                  27      2    93%   42-43
/mnt/d/github/PySyft/src/syft/decorators/typecheck.py                                                            42      5    88%   12-15, 86, 97, 109
/